In [2]:
import os
import nvtabular as nvt
from nvtabular.ops import *

from merlin.models.utils.dataset import unique_rows_by_features
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.schema.tags import Tags
import pandas as pd 
import tensorflow as tf

import merlin.models.tf as mm
from merlin.io.dataset import Dataset

2024-01-12 06:32:24.293977: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2024-01-12 06:32:27.136700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-12 06:32:27.139078: I tensorflow/compiler/xla/stream_execut

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.10/dist-packages/merlin_sok-1.2.0-py3.10-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.10/dist-packages/merlin_sok-1.2.0-py3.10-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2024-01-12 06:32:30.534765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-12 06:32:30.536787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-12 06:32:30.538478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
dataset_path = '/home/jupyter/merlin_rec_sys/data/toy_dataset.parquet'
toy = nvt.Dataset(dataset_path)

In [3]:
toy_embeddings = [f'{idx}' for idx in range(18,402)]
toy_feedback = [f'{idx}' for idx in range(402,733)]

In [4]:
#### Workflow 
userid = ['0'] >> Categorify() >> TagAsUserID()
itemid = ['17'] >> Categorify() >> TagAsItemID()

item_feats_1 = ['9','10','11','12','13','14','15','16'] >> Normalize() >> TagAsItemFeatures()
item_feats_2 = ['8'] >> Categorify() >> TagAsItemFeatures()

user_feats_1 = (['1','3','4','7']
    >> Categorify()
    >> TagAsUserFeatures())

embeddings = toy_embeddings >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS]) >> TagAsItemFeatures()

user_feats_2 = toy_feedback >> Categorify() >> TagAsUserFeatures()

user_feats_3 = ['2','5','6'] >> Normalize() >> TagAsUserFeatures()

outputs = userid + itemid + item_feats_1 + item_feats_2 + user_feats_1 + embeddings + user_feats_2 + user_feats_3

In [5]:
output_dir = '/home/jupyter/merlin_rec_sys/data/'
output_path = os.path.join(output_dir, "processed_data")

In [6]:
workflow = nvt.Workflow(outputs)

In [7]:
workflow.fit(toy)
workflow.transform(toy).to_parquet(
    output_path=output_path + "/train/",
    out_files_per_proc = 1)

In [ ]:
# workflow.save('/home/jupyter/merlin_rec_sys/data/toy_workflow')

In [8]:
train = Dataset(os.path.join(output_dir, "processed_data", "*.parquet"),part_size='128MB')

/usr/local/lib/python3.10/dist-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (129840000) (bytes) of parquet file is bigger than requested part_size (128000000) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(


In [9]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
train.schema = schema

In [10]:
# create user schema using USER tag
user_schema = schema.select_by_tag(Tags.USER)
# create user (query) tower input block
user_inputs = mm.InputBlockV2(user_schema)
# create user (query) encoder block
query = mm.Encoder(user_inputs, mm.MLPBlock([128,64], no_activation_last_layer=True))

# create item schema using ITEM tag
item_schema = schema.select_by_tag(Tags.ITEM)
# create item (candidate) tower input block
item_inputs = mm.InputBlockV2(item_schema)
# create item (candidate) encoder block
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128,64], no_activation_last_layer=True))

In [ ]:
model = mm.TwoTowerModelV2(query, candidate)
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10)])
model.fit(train, batch_size=8192, epochs=1)

2024-01-12 05:32:47.076147: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


674/674 [==============================] - 1323s 2s/step - loss: 8.8668 - recall_at_10: 0.0043 - ndcg_at_10: 0.0023 - regularization_loss: 0.0000e+00 - loss_batch: 8.8610


In [12]:
query_tower = model.query_encoder
query_tower.save(os.path.join('/home/jupyter/merlin_rec_sys/data/', "toy_query_tower"))

INFO:tensorflow:Assets written to: /home/jupyter/merlin_rec_sys/data/toy_query_tower/assets


INFO:tensorflow:Assets written to: /home/jupyter/merlin_rec_sys/data/toy_query_tower/assets


In [13]:
### Here we are loading the query tower. 
query_tower = tf.keras.models.load_model('/home/jupyter/merlin_rec_sys/data/toy_query_tower',compile=False)

model = mm.TwoTowerModelV2(query_tower, candidate)
model.compile()

candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)

topk_model = model.to_top_k_encoder(candidate_features, k=10, batch_size=1)

INFO:tensorflow:Assets written to: /tmp/tmpqxs0pdyx/assets


INFO:tensorflow:Assets written to: /tmp/tmpqxs0pdyx/assets


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /usr/local/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory

## Here are the errors I get w/ a restarted kernel and I load in the query tower

In [3]:
train = Dataset(os.path.join('/home/jupyter/merlin_rec_sys/data/test', "train", "*.parquet"),part_size='128MB')

/usr/local/lib/python3.10/dist-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (129840000) (bytes) of parquet file is bigger than requested part_size (128000000) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(


In [4]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
train.schema = schema

In [5]:
# create item schema using ITEM tag
item_schema = schema.select_by_tag(Tags.ITEM)
# create item (candidate) tower input block
item_inputs = mm.InputBlockV2(item_schema)
# create item (candidate) encoder block
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128,64], no_activation_last_layer=True))

In [6]:
query_tower = tf.keras.models.load_model('/home/jupyter/merlin_rec_sys/data/toy_query_tower',compile=False)

In [7]:
model = mm.TwoTowerModelV2(query_tower, candidate)
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10)])

In [8]:
candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)

topk_model = model.to_top_k_encoder(candidate_features, k=10, batch_size=1)

  (blocks): List(
    (0): ParallelBlock(
      (_aggregation): ConcatFeatures()
      (parallel_layers): Dict(
        (categorical): ParallelBlock(
          (parallel_layers): Dict(
            (17): EmbeddingTable(
              (features): Dict(
                (17): ColumnSchema(name='17', tags={<Tags.ID: 'id'>, <Tags.ITEM: 'item'>, <Tags.CATEGORICAL: 'categorical'>}, properties={'num_buckets': None, 'freq_threshold': 0.0, 'max_size': 0.0, 'cat_path': './/categories/unique.17.parquet', 'embedding_sizes': {'cardinality': 1378202.0, 'dimension': 512.0}, 'domain': {'min': 0, 'max': 1378201, 'name': '17'}}, dtype=DType(name='int64', element_type=<ElementType.Int: 'int'>, element_size=64, element_unit=None, signed=True, shape=Shape(dims=(Dimension(min=0, max=None),))), is_list=False, is_ragged=False)
              )
              (table): Embedding()
            )
            (8): EmbeddingTable(
              (features): Dict(
                (8): ColumnSchema(name='8', tags={<Tags.I

ValueError: Model Encoder(
  (blocks): List(
    (0): ParallelBlock(
      (_aggregation): ConcatFeatures()
      (parallel_layers): Dict(
        (categorical): ParallelBlock(
          (parallel_layers): Dict(
            (17): EmbeddingTable(
              (features): Dict(
                (17): ColumnSchema(name='17', tags={<Tags.ID: 'id'>, <Tags.ITEM: 'item'>, <Tags.CATEGORICAL: 'categorical'>}, properties={'num_buckets': None, 'freq_threshold': 0.0, 'max_size': 0.0, 'cat_path': './/categories/unique.17.parquet', 'embedding_sizes': {'cardinality': 1378202.0, 'dimension': 512.0}, 'domain': {'min': 0, 'max': 1378201, 'name': '17'}}, dtype=DType(name='int64', element_type=<ElementType.Int: 'int'>, element_size=64, element_unit=None, signed=True, shape=Shape(dims=(Dimension(min=0, max=None),))), is_list=False, is_ragged=False)
              )
              (table): Embedding()
            )
            (8): EmbeddingTable(
              (features): Dict(
                (8): ColumnSch